In [ ]:
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 16.2MB 8.1MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp37-none-any.whl size=16172935 sha256=b41abb36b7bd584c1ffd9b3deef847273daa52039098fce26ed04f5ba148ad9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-y1tfxrka/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

import spacy
import es_core_news_sm

import codecs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/TFG/textos.csv'

# Leer csv donde se encuentran el conjunto de textos de los distintos ODS, cada uno de los atributos esta delimitado por ;; 
df = pd.read_csv(path, sep=";;", engine='python', names=['Numero texto', 'ODS', 'Texto'])

# Poner los ODS con tipo int 
for i in range(len(df.index)):
  df['ODS'][i] = df['ODS'][i][3:]
df

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Numero texto,ODS,Texto
0,1001,1,﻿﻿﻿﻿EROSKI y sus clientes han logrado canaliz...
1,1002,1,﻿﻿﻿﻿ El Corte Inglés ha recogido en 2020 casi...
2,1003,1,﻿﻿﻿﻿Fundación Solidaridad Carrefour ha presen...
3,1004,1,﻿﻿﻿﻿La cadena de hoteles RIU se ha unido una ...
4,1005,1,﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿Orange pondrá en marcha el próxim...
...,...,...,...
9949,10953,17,"Durante el Thales iDay, se llevó a cabo una m..."
9950,10954,17,2016 ha sido un año repleto de desafíos para ...
9951,10955,17,Con esta firma SUPRACAFÉ se compromete a impl...
9952,10956,17,En la página web del Congreso ya está abierto...


In [ ]:
# Función que elimina los caracteres que tienen algunos textos del csv al principio 
def clear_first_token(token):
  result = ""
  char = '\ufeff'
  for i in range(len(token)):
    if token[i] != char: result += token[i]
  return result

def preprocess(text):
  # Texto en minusculas
  text_lower = text.lower()
  # Tokenize 
  token_word = nltk.word_tokenize(text_lower, "spanish")
  # Algunos textos tienen caracteres raros al principio, por tanto hay que eliminarlos para que no influyan en el código 
  print(token_word)
  token_word[0] = clear_first_token(token_word[0])
  # Stopwords de palabras españolas
  stopword_spanish = stopwords.words("spanish")
  i = 0

  while(i < len(token_word)):
    # Se eliminan los tokens que se encuentren dentro de las stopwords 
    if token_word[i] in stopword_spanish:
      token_word.remove(token_word[i])
    #Se elimina cualquier token que sea distinto de caracteres alfanumericos
    elif not (token_word[i].isalpha()):
      token_word.remove(token_word[i])
    else:
      # Debido a que la libreria nltk no lematiza textos en español es necesario utilizar otra libreria de NLP la cual es Spacy
      word = sp(token_word[i])[0]
      token_word[i] = word.lemma_
      i = i+1
  return token_word

In [ ]:
# Tokenize de todos los textos del csv 
sp = es_core_news_sm.load()
df['Tokens']=df['Texto'].apply(preprocess)


In [ ]:
# Se guardan los textos tokenizados en otro csv 
df
df.to_csv("textos_procesados.csv")
